In [21]:
%matplotlib inline
import os
import pandas as pd
import fnmatch
import numpy as np
import scipy as sp
import scipy.sparse
import matplotlib.pyplot as plt

def interp(t_obs,f_obs,h=0.5):
    T = t_obs[-1]-t_obs[0]
    T_f=t_obs[-1]
#    print(T,t_obs[-1],t_obs[0])
    n = int(np.floor(T/h-1))
#    print("n", n)
    t = np.arange(t_obs[0]+h,T_f+h,h)
#    print(t[:4],t[-3:])
    time_indices = [np.argwhere(np.abs(t-time)<1e-8)[0,0] for time in t_obs if time>t_obs[0]]
#    print len(time_indices),len(t_obs)
    projection = np.eye(n+1)[time_indices]
    e = np.ones(n+2)
    F = sp.sparse.spdiags([1*e,-4*e,3*e],[-2,-1,0],n+1,n+1).todense()/2/h
    F[0,0],F[0,1]=-1/h,0.5/h
    F[1,0],F[1,1],F[1,2]=-.8/h,.6/h,.2/h
    invert = np.linalg.inv(np.dot(np.dot(projection,np.linalg.inv(np.dot(F.T,F))),projection.T))
    total = np.dot(np.dot(np.linalg.inv(np.dot(F.T,F)),projection.T),invert)
    return t,f_obs[0]+np.array(np.dot(total,f_obs[1:]-f_obs[0]).T).reshape(n+1)

def fat_func_maker(par):
    l0 = par[0]
    l2 = par[1]
    x2 = par[2]
    cf = par[3]
    xcl = par[4]
    def lipo_func(x_try):
        return l0 + l2/(1+(x_try/x2)**2)
    def clear_func(x_try):
        y = (x_try/xcl)**2
        return cf*(1.0 + y/(1+y))
    return lipo_func,clear_func

def xf(parx,ins_interp,h=0.5):
    cx = parx[0]
    Ibx = parx[1]
    n = np.shape(ins_interp)[0]
    e = np.ones(n)
    F = sp.sparse.spdiags([1*e,-4*e,3*e],[-2,-1,0],n,n).todense()/2/h
    F[0,0],F[0,1]=-1/h,0.5/h
    F[1,0],F[1,1],F[1,2]=-.8/h,.6/h,.2/h
    F = F + cx*np.diag(e)
    x = np.dot(np.linalg.inv(F),cx*(ins_interp-Ibx)).reshape(ins_interp.shape)
    x = np.maximum(x,np.zeros(n))
    return np.array(x).reshape(ins_interp.shape)

def ff(par,f_init,x,h=0.5):
    lip_fun,clear_fun = fat_func_maker(par)
    n = np.shape(x)[0]-1
    e = np.ones(n+1)
    F = sp.sparse.spdiags([1*e,-4*e,3*e],[-2,-1,0],n+1,n+1).todense()/2/h
    F[0,0],F[0,1]=-1/h,0.5/h
    F[1,0],F[1,1],F[1,2]=-.8/h,.6/h,.2/h
    F = F + np.diag(clear_fun(x))
    f= np.maximum(np.dot(np.linalg.inv(F),(lip_fun(x)-f_init*clear_fun(x))).reshape(x.shape)+f_init,np.zeros(n+1))
##    plt.plot(np.arange(n+1),f.T)
##    plt.show()
    return np.array(f).reshape(x.shape)
    
def gf(parg,g_init,x,h=0.5):
    sg = parg[0]
    si = parg[1]
    gb = parg[2]
    n = np.shape(x)[0]-1
    e = np.ones(n+1)
    F = sp.sparse.spdiags([1*e,-4*e,3*e],[-2,-1,0],n+1,n+1).todense()/2/h
    F[0,0],F[0,1]=-1/h,0.5/h
    F[1,0],F[1,1],F[1,2]=-.8/h,.6/h,.2/h
    F = F + np.diag((sg+si*x))
    g = np.maximum(np.dot(np.linalg.inv(F),(sg*(gb-g_init)-si*x*g_init)).reshape(x.shape)+g_init,np.zeros(n+1))
##    plt.plot(np.arange(n+1),g.T)
##    plt.show()
    return np.array(g).reshape(x.shape)


location=os.getcwd()

# subjects_final=open('subjects_final.txt','w')
# subjects_final.write('subject' +\
#                '\t' + 'Start/Finish' +\
#                '\t' + 'full/partial' +\
#                '\t' +'cx' +\
#                '\t' + 'Ibx' +\
#                '\t' + 'l0' +\
#                '\t' + 'l1' +\
#                '\t' + 'x2' +\
#                '\t' + 'cf' +\
#                '\t' + 'xcl' +\
#                '\t' + 'SG' +\
#                '\t' + 'SI' +\
#                '\t' + 'Gb' +\
#                '\n')

full=r'subjects_final.xlsx'
df = pd.read_excel(full)
par=df.values[np.where(df['subject'] == 1)[0][0],]
subject_id=par[0]
index=par[1]
for file in os.listdir(location):
    if fnmatch.fnmatch(file, '%s %s*.xlsx' % (subject_id, index)):
        i=13
#         df = pd.read_excel(file)
#         Time = (df['Time'][i:23].values).astype(dtype='float64')
#         Glucose = (df['Glucose (mg/dL)'][i:23].values).astype(dtype='float64')
#         Insulin = (df['Insulin (uU/mL)'][i:23].values).astype(dtype='float64')
#         C = (df['C-peptide (ng/mL)'][i:23].values).astype(dtype='float64')
#         F = (df['FFA (mEq/L)'][i:23].values).astype(dtype='float64')
#         num_data = np.vstack((Time, Glucose, Insulin, C, F)).T
#         num_data = np.delete(num_data, np.where(num_data == np.float64(-1))[0], axis=0)

#         t_obs = num_data[:,0]
#         g_obs = num_data[:,1]
#         i_obs = num_data[:,2]
#         c_obs = num_data[:,3]
#         f_obs = num_data[:,4]
#         t_interp,ins = interp(t_obs,i_obs)

#         h=0.5
#         T = t_obs[-1]-t_obs[0]
#         T_f=t_obs[-1]
#         n = int(np.floor(T/h-1))
#         t = np.arange(t_obs[0]+h,T_f+h,h)
#         parx = par[3:5] +[0, 0]
#         parf = par[5:10]+[0, 0, 0, 0, 0]
#         parg = par[10:13]+[0, 0, 0]
# #         parx = [0.1639, 4.672]
# #         parf = [0.0025146, 0.030109, 8.7594, 0.069251, 7.89024]
# #         parg = [0.00346352, 0.000206, 151]
#         x = xf(parx,ins)
#         f = ff(parf,f_obs[0],x)
#         g = gf(parg,g_obs[0],x)

#         time_indices = [np.argwhere(np.abs(t-time)<1e-8)[0,0] for time in t_obs if time>t_obs[0]]
#         g_fit = g[time_indices]
#         f_fit = f[time_indices]

#         g_cost=np.mean(abs(g_obs[1:]-g_fit))
#         f_cost=np.mean(abs(f_obs[1:]-f_fit)*18.018)
#         cost_full = np.mean([g_cost, f_cost])

#         plt.rcParams.update({'font.size': 18})
#         plt.rcParams.update({'lines.markersize':10})
#         fig_full, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
#         plt.subplots_adjust(wspace=0.5, bottom=0.18)

#         ax1.plot(t, g)
#         ax1.plot(t_obs, g_obs, 'ro')
#         ax1.set_xlabel('Time')
#         ax1.set_ylabel('Glucose')

#         ax2.plot(t, f)
#         ax2.plot(t_obs, f_obs, 'ro')
#         ax2.set_xlabel('Time')
#         ax2.set_ylabel('FFA')

In [24]:
int(subject_id)

1

In [14]:
par

array([1, 'Final', 'partial', 0.0251714383923879, 3.24300503387576,
       0.00231066139898211, 0.131456582808071, 9.63353905963842,
       0.0265562229459352, 28.296542318778, 0.00590723229773331,
       7.55814569211189e-05, 84.4298937136545], dtype=object)